In [2]:
import numpy as np

from operator import add
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext

from pyspark.ml.feature import RegexTokenizer, CountVectorizer
from pyspark.ml.feature import StopWordsRemover, VectorAssembler
from pyspark.ml.feature import IDF
from pyspark.ml import Pipeline, PipelineModel

from pyspark.sql.functions import *
from pyspark.sql.types import *


sc = SparkSession.builder.getOrCreate()
spark = SparkSession.builder.appName('FinalProject').getOrCreate()

# Get sparkcontext from 
sqlContext = SQLContext(sc)

df_business = spark.read.json('/Volumes/Nischay HDD/yelp_academic_dataset_business.json')

/Users/nischay/spark/python/pyspark/sql/context.py:77: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(
22/03/31 23:31:55 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [4]:
df_business_MA = df_business.select('business_id', 'name',  'address', 'city', \
                                      'state', 'postal_code', 'latitude', 'longitude', 'stars', 'review_count', 'categories') \
                                .filter( (df_business.city == 'Vancouver') & (df_business.categories.contains('Restaurants') ) )

business_count = df_business_MA.count()
print(f'MA Business count: {business_count}')

# Sampling data to run on local
sampled_business_MA = df_business_MA.sample(0.05)

MA Business count: 5026


In [5]:
sampled_business_MA.write.format('jdbc').options(
      url='jdbc:mysql://localhost/food_hunter_development',
      driver='com.mysql.jdbc.Driver',
      dbtable='restaurants',
      user='root',
      password='Nanu1996').mode('append').save()

Loading class `com.mysql.jdbc.Driver'. This is deprecated. The new driver class is `com.mysql.cj.jdbc.Driver'. The driver is automatically registered via the SPI and manual loading of the driver class is generally unnecessary.


In [1]:
spark.stop()

In [6]:
sampled_business_MA.count()

2483

In [ ]:
# SCRIPT YOU NEED 


# ERADING ALL JSON FROM LOCAL

df_business = spark.read.json('/Volumes/Nischay HDD/yelp_academic_dataset_business.json')
df_reviews  = spark.read.json('/Volumes/Nischay HDD/yelp_academic_dataset_review.json')
df_users  = spark.read.json('/Volumes/Nischay HDD/yelp_academic_dataset_user.json')


# FILTERING DATASET TO VANCOUVER

df_business_MA = df_business.select('business_id', 'name',  'address', 'city', \
                                      'state', 'postal_code', 'latitude', 'longitude', 'stars', 'review_count', 'categories') \
                                .filter( (df_business.city == 'Vancouver') & (df_business.categories.contains('Restaurants') ) )

business_count = df_business_MA.count()
print(f'MA Business count: {business_count}')

# Sampling data to run on local
sampled_business_MA = df_business_MA.sample(0.05)


# FETCHING USER REVIEWS FOR FILTERED BUSINESES

df_reviews_MA = df_reviews.join(sampled_business_MA, on = 'business_id', how = 'inner') \
                          .select(df_reviews.business_id, df_reviews.user_id, df_reviews.review_id, df_reviews.stars)
df_reviews_count = df_reviews_MA.count()
print(f'Business Reviews count: {df_reviews_count}')



# FETCHING USERS
df_users_MA = df_users.join(df_reviews_MA, on = 'user_id', how = 'inner') \
                          .select(df_users.user_id, df_users.name, df_users.review_count, df_users.yelping_since, \
                                  df_users.useful, df_users.funny , df_users.cool , df_users.fans , df_users.average_stars)
df_users_MA.show(2)

df_users_MA_count = df_users_MA.count()
print(f'Users count: {df_users_MA_count}')




# WRITRING THE DFs into paraquet

sampled_business_MA.coalesce(1).write.parquet('New_Small_Datasets/small_business_datset')
df_reviews_MA.coalesce(1).write.parquet('New_Small_Datasets/small_reviews_dataset')
df_users_MA.coalesce(1).write.parquet('New_Small_Datasets/small_users_dataset')







MA Business count: 5026


Business Reviews count: 16472


In [ ]:
# Read data generated from above process
business_df = spark.read.parquet('New_Small_Datasets/small_business_datset')
business_count = business_df.count()

reviews_df = spark.read.parquet('New_Small_Datasets/small_reviews_dataset')
reviews_count = reviews_df.count()

users_df = spark.read.parquet('New_Small_Datasets/small_users_dataset')
users_count = users_df.count()

print(f'Business COunt: {business_count}')
print(f'Reviews COunt: {reviews_count}')
print(f'Users COunt: {users_count}')